# Computational Theory - Shane Walsh - G00406694

# Task 1: Binary Representations

## Binary Representations
So for this task we are required to create various python functions and demonstrate their use with examples and tests. There are four functions in total, two focus on the process of rotating bits in a 32-bit unsigned integer to the left and right places. Another explores the implementation of a function that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.
The last function focuses on developing a function that takes a majority vote of the bits in x, y and z.

## 1. The function rotl(x, n=1)

This function goes through a process of rotating bits in a 32-bit unsigned integer to a direction. In this case, to the left by a set number (such as 1 or 2) of positions. To implement this, we first need to understand the syntax for Bitwise shifting and what it is.

Bitwise shifting is a fundamental operation in computational theory that involves the manipulation of data at the binary level, like we're doing for these functions. 

At its core, it involves moving all the bits in a binary number left or right by a specified degree of positions. This primarily involves two different operations: 

### Bitwise left shift: <<

So when left shifting, each bit in the binary string moves n positions to the left, the positions farthest to the right are then replaced with zeros. 

### Bitwise right shift: >>

For right shifting, its basically the same process inverted. Each bit moves n positions right, and the positions fartest to the left are filled with zeros. 

Bitwise shifting is a fundamental operation described in detail in:
- [Bit Manipulation - Stanford CS Education Library](https://graphics.stanford.edu/~seander/bithacks.html)
- [Bitwise Operations - Computer Systems: A Programmer's Perspective](https://dl.ebooksworld.ir/books/Computer.Systems.A.Programmers.Perspective.3rd.Edition.Pearson.9780134092669.EBooksWorld.ir.pdf)

In [156]:
# Bitwise shifting of a number
x = 0b1100
print("Base value of X: " + bin(x)) # Base value
print("Shifted to the left by 2: " + bin(x << 2)) # Shift left by 2
print("Shifted to the right by 2: " + bin(x >> 2)) # Shift right by 2


Base value of X: 0b1100
Shifted to the left by 2: 0b110000
Shifted to the right by 2: 0b11


A fun little note, left-shifting by 1 is equal to multiplying by 2, while a shift of 2 is equal to multiplying by 4. This continues upwards linearly, by 2^n.

This is similar with left-shifting but division instead. 1 bit shift right is equal to dividing by 2 and continues upwards, by 2^n.

### Masking

Now in my opinion its important that to avoid losing any bits during the shift process, we use what's known as masking. This ensures that our result doesn't grow larger by mistake and maintains the 32 bit size of my result.

As seen in the example below, we use a 32-bit constraint using "& 0xFFFFFFFF" with our number to mask it. This is crucial for our bit rotations. 

In [157]:
# Masking in action
large_number = 0xFFFFFFFF + 1  # This would be 2^32, exceeding 32 bits
masked_number = large_number & 0xFFFFFFFF
print(f"\n32-bit overflow example:")
print(f"Number exceeding 32 bits: {hex(large_number)} (33 bits)")
print(f"After applying 32-bit mask: {hex(masked_number)} (32 bits)")


32-bit overflow example:
Number exceeding 32 bits: 0x100000000 (33 bits)
After applying 32-bit mask: 0x0 (32 bits)


### Bitwise Operations
The final step to tie this together is by using bitwise operations, which are your classic AND, OR, NOT and XOR gates. These allow us to control our results dynamically by manipulating individual bits. 

Personally I think these are great for efficiency, typically acting faster than normal arithmetic operations. They allow for direct manipulation of memory and registers and are able to store multiple boolean valuables for singular integers. Within Cryptography, they form the basis of many algorithms such as SHA-256, as we'll see later. 

These bitwise operations form the foundation of digital logic and are explained in:
-  [*Digital Design and Computer Architecture* (3rd ed.), Chapter 1.4: Boolean Algebra and Logic Gates.](https://allbooksfordownloading.wordpress.com/wp-content/uploads/2017/01/digital-design-and-computer-architecture-by-david-and-sarah-harris.pdf)
- Python Documentation: [Bitwise Operations](https://docs.python.org/3/library/stdtypes.html#bitwise-operations-on-integer-types)

In [158]:
# Examples of bitwise operations
x = 0b1100 # Binary representation of 12
y = 0b1010 # Binary representation of 10

# OR - bits are set if either bit is set
result_or = x | y
print(f"x | y: {bin(result_or)} (decimal: {result_or})")

# AND - bits are set if both bits are set
result_and = x & y
print(f"x & y: {bin(result_and)} (decimal: {result_and})")

# XOR - bits are set if only one bit is set
result_xor = x ^ y
print(f"x ^ y: {bin(result_xor)} (decimal: {result_xor})")

# NOT - bits are inverted
result_not = ~x
print(f"~x: {bin(result_not)} (decimal: {result_not})")

x | y: 0b1110 (decimal: 14)
x & y: 0b1000 (decimal: 8)
x ^ y: 0b110 (decimal: 6)
~x: -0b1101 (decimal: -13)


In the end our rotation function looks like the following. It shifts bits left by n positions with (x << n). It then shifts bits that would be lost when shifting left, to the right with (x >> (32 - n)). We put an OR operator between these to combine them. Lastly, we use an AND operator to bring in our masking for ensuring the result stays at 32 bits. 

In [159]:
import numpy as np
import math

def rotl(x, n=1):
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF # << shifts the bits to left, >> shifts the bits to right.

x = 0b1100
print(f'rotl(0b1100, 2): {bin(rotl(x, 2))}')
print(f'rotl(0b1100, 3): {bin(rotl(x, 3))}')

rotl(0b1100, 2): 0b110000
rotl(0b1100, 3): 0b1100000


### Demonstrating the use of Rotl(x, n=1) 

In [160]:
x = 0b10101010  # 170 in decimal
n = 4

print(f"Demonstrating rotl({bin(x)}, {n})")
print(rotl(x, n))

# Additional examples with different n values
print(f"Demonstrating rotl({bin(x)}, 1) - Single bit rotation")
print(rotl(x, 1))  # Rotate by 1 bit

print(f"Demonstrating rotl with a larger number")
large_num = 0xABCDEF12  # A larger hex number
print(rotl(large_num, 8))  # Rotate by a byte

Demonstrating rotl(0b10101010, 4)
2720
Demonstrating rotl(0b10101010, 1) - Single bit rotation
340
Demonstrating rotl with a larger number
3454997163


## 2. The function rotr(x, n=1)

Rotr largely works the same as rotl in its construction, just with bitwise operations inverted to go the opposite direction. 

In [161]:
def rotr(x, n=1):
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

x = 0b1100

# Test the function.
print(f'rotr(0b1100, 2): {bin(rotr(x, 2))}')

rotr(0b1100, 2): 0b11


### Applications of Bit Shifts and Rotation
The bit shifting and rotations seen in these functions is used in various aspects of technology. As mentioned, bit rotation is fundamental to Cryptography. They're both also used for graphics processing, particularly image manipulation and they're used in embedded systems when resources for the hardware is quite limited. 

The bit rotation functions implemented here are similar to those used in cryptographic algorithms:
- NIST (2015). *FIPS PUB 180-4: Secure Hash Standard (SHS)*. National Institute of Standards and Technology. [https://doi.org/10.6028/NIST.FIPS.180-4](https://doi.org/10.6028/NIST.FIPS.180-4) 
- *RFC 3174: US Secure Hash Algorithm 1 (SHA1)*, Section 3: Operations on Words. [https://tools.ietf.org/html/rfc3174#section-3](https://tools.ietf.org/html/rfc3174#section-3)

## 3. The function ch(x, y, z)

For this function we need to make use of bit selection logic using bitwise operators on our chosen binary values. In this particular implementation: 

- For each bit position of x being 1, we choose the associated or corresponding bit from y.
- For each bit position of x being 0, we choose the corresponding bit from z.

This is an important concept in my opinion, its used in cryptographic hash functions like SHA-256 and the like. Its often referred to as a nonlinear mixing function.

Let me further break down how to accomplish this: 
- Using (x & y) will take bits from y where x has 1s.
- Using  (~x & z) inverts the x and then performs an AND bitwise operation with z. This keeps only bits where x has 0s.
- Lastly, the use of ^ (XOR) combines the results from both, giving us a mix of bits from y and z based on the values from x. 

The ch function is a core component of SHA-256 and other cryptographic hash functions:
- NIST (2015). *FIPS PUB 180-4: Secure Hash Standard (SHS)*, Section 4.1.2: SHA-256 Functions. [https://doi.org/10.6028/NIST.FIPS.180-4](https://doi.org/10.6028/NIST.FIPS.180-4)
- *Understanding Cryptography: A Textbook for Students and Practitioners*, Chapter 11: Hash Functions. [https://doi.org/10.1007/978-3-642-04101-3](https://doi.org/10.1007/978-3-642-04101-3)

In [162]:
def ch(x, y, z): # choose between x, y, z. 
    return (x & y) ^ (~x & z) # & is bitwise AND, ^ is bitwise XOR, ~ is bitwise NOT. So this returns x AND y XOR NOT x AND z.

x = 0b1100
y = 0b1010
z = 0b1001

print(f'ch(0b1100, 0b1010, 0b1001): {bin(ch(x, y, z))}')

ch(0b1100, 0b1010, 0b1001): 0b1001


In [163]:
# Example of using the ch function with different values
x = 0b1111
y = 0b0000
z = 0b1111
print(f"ch(0b1111, 0b0000, 0b1111): {bin(ch(x, y, z))}") # Should return 0b0000

x = 0b1010
y = 0b1100
z = 0b0110
print(f"ch(0b1010, 0b1100, 0b0110): {bin(ch(x, y, z))}") # Should return 0b1100

ch(0b1111, 0b0000, 0b1111): 0b0
ch(0b1010, 0b1100, 0b0110): 0b1100


## 4. The function maj(x, y, z)

This function is akin to a voting system through boolean logic. It returns 1 if and only if more than half of the inputs are 1. In this case, since we have three inputs, at least two must be 1.

Breaking this down:
- Using (x & y), we set bits where both x AND y have 1s present.
- Using (x & z), we set bits where both x AND z have 1s.
- Using (y & z), we set bits where both y AND z have 1s.
Lastly, these are XORed together to keep it as a majority logic.

The majority function is seen across SHA-256 and other secure hash algorithms for its non-linear behaviour that's great for resisting crypto analysis, for its balanced nature and for the Avalache Effect: where small input changes can lead to extensive output changes. Its very worth exploring how its implemented I think.

The majority function (maj) is specified in the SHA-256 algorithm:
- NIST (2015). *FIPS PUB 180-4: Secure Hash Standard (SHS)*, Section 4.1.2: SHA-256 Functions. [https://doi.org/10.6028/NIST.FIPS.180-4](https://doi.org/10.6028/NIST.FIPS.180-4)
- *SHA-3 Standard: Permutation-Based Hash and Extendable-Output Functions*. [https://doi.org/10.6028/NIST.FIPS.202](https://doi.org/10.6028/NIST.FIPS.202)

In [164]:
def maj(x, y, z): # Takes a majority vote of x, y, z.
    return (x & y) ^ (x & z) ^ (y & z) # ^ means XOR. So this returns x AND y XOR x AND z XOR y AND z.

x = 0b1100
y = 0b1010
z = 0b1001

x1 = 0b1100
y1 = 0b1010
z1 = 0b1010

x2 = 0b1111
y2 = 0b0000
z2 = 0b1111

# Examples
print(f'maj(0b1100, 0b1010, 0b1001): {bin(maj(x, y, z))}')
print(f'maj(0b1100, 0b1010, 0b1010): {bin(maj(x1, y1, z1))}')
print(f'maj(0b1111, 0b0000, 0b1111): {bin(maj(x2, y2, z2))}')

maj(0b1100, 0b1010, 0b1001): 0b1000
maj(0b1100, 0b1010, 0b1010): 0b1010
maj(0b1111, 0b0000, 0b1111): 0b1111


# Task 2: Hash Functions

### What are Hash Functions?

They are algorithms that transform input data from any arbitrary size into a fixed-size value instead. This will usually involve a string of characters or a number. The process is deterministic in nature, which means the same input will always grant you the same output. 
Adjacent to this, you could also say its collision resistant, where different inputs should rarely if ever grant you the same output. 
Generally its fast to compute and the distribution of values is spread evenly across the total range of possibilities. 

- *Introduction to Algorithms* (3rd ed.), Chapter 11: Hash Tables. MIT Press. [https://mitpress.mit.edu/books/introduction-algorithms-third-edition](https://mitpress.mit.edu/books/introduction-algorithms-third-edition)
- *NIST Special Publication 800-107: Recommendation for Applications Using Approved Hash Algorithms*. [https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-107r1.pdf](https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-107r1.pdf)

### Conversion From C

I'll now convert the [specified hash function from the C Programming language](https://github.com/ianmcloughlin/computational_theory/blob/main/assessment/tasks.md) to python. This will require quite a few changes to the code but in my opinion it should be doable. Luckily, we don't need to declare variable types. While the hash function in C uses pointer's and null terminator checking, Python makes this much simpler as we can just iterate directly over the string. The Hash function that we're converting in this process is from page 128 - 129 of [The C Programming Language](https://seriouscomputerist.atariverse.com/media/pdf/book/C%20Programming%20Language%20-%202nd%20Edition%20(OCR).pdf) by Brian Kernighan and Dennis Ritchie.

In [165]:
def Hash_string(s): # No need to declare var, just use it with the input, no pointer.
    hashval = 0 # No use of 'unsigned' in Python.
    # Iterate directly over string.
    for c in s:
        hashval = ord(c) + 31 * hashval # ord() returns ASCII value of char, 31 is prime number, good for hashing.
    return hashval % 101 # Modulus is unchanged, 101 is prime number, reduces collisions.

The final conversion would look like this. Rather than dereferencing the character pointer in C, we can use Python's [ord() function](https://docs.python.org/3.4/library/functions.html#ord) to pull out the relevant [ASCII value](https://www.ascii-code.com/) of a character. The modulus element on the hashVal has no necessary changes.

### Using Prime Numbers in Hash

The numbers 31 and 101 are prime numbers. To my knowledge, using prime numbers for hash values gives us a smoother distribution. 31 is proven to produce very few collisions with English text.
101 helps determine our hash table size and reduce collisions which can quite easily come up. Its small enough that it'll still be efficient in regards to memory usage but large enough for collision reduction.

- [*The Art of Computer Programming, Volume 3: Sorting and Searching* (2nd ed.), Section 6.4: Hashing. Addison-Wesley.](https://www-cs-faculty.stanford.edu/~knuth/taocp.html)
- [*Hash function related summations*. Journal of Algorithms, 64(1), 41-50. ](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9118906)
- [*Hash Functions for Hash Table Lookup*. ](http://www.burtleburtle.net/bob/hash/doobs.html)

In [166]:
# Testing Hash function
print(f'Hash_string("hello"): {Hash_string("hello")}')
print(f'Hash_string("world"): {Hash_string("world")}')
print(f'Hash_string("hello world"): {Hash_string("hello world")}')

Hash_string("hello"): 17
Hash_string("world"): 34
Hash_string("hello world"): 13


### Some Background
The K&R hash function (Kernighan and Ritchie) function comes from their book first published in 1978. This book is seminal for practically defining the C programming language and has many practical algorithms such as this hash function present within its pages. It was designed for a time when resources were more limited, its simple but efficient nature making it valuable in my opinion. Alternatives to this hash mainly include SHA-256 and Blake-2, both used for Cryptographic security. 

- [**SHA-256**: Designed by NSA, standardized by NIST in FIPS 180-4.](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)
- [**BLAKE2**: High-performance cryptographic hash function.](https://www.blake2.net/)
- [*The First 30 Years of Cryptographic Hash Functions and the NIST SHA-3 Competition*. In Topics in Cryptology – CT-RSA 2010.](https://doi.org/10.1007/978-3-642-11925-5_1)

# Task 3: SHA256

So for this task, we need to calculate the SHA256 padding for a file supplied in the method as input. Ideally, I should adhere to the [NIST FIPS 180-4 specification](https://doi.org/10.6028/NIST.FIPS.180-4) during this process. In the end I'll be returning the padding as a collection of bytes.

### What is SHA256?
It stands for Secure Hash Algorithm. Its a cryptographic hash function from the SHA-2 family which were developed by the National Security Agency (NSA). Publised in 2001 by the National Institute of Standards and Technology (NIST), it makes a fixed-size 256 bit (which is 32 bytes) hash value from any inputted data of a fairly arbitrary size. 

In relation to this implementation, padding is a part of the larger design of SHA256, but its very necessary in my opinion.
- SHA256 processes data in these fixed-size blocks of 512 bits (64 bytes). For this, any message must be padded so that its length is also a multiple of 512 bits. 
- You can include the original length of the message in the padding to aid in certain cryptographic attacks (like extension attacks).
- Padding ensures that messages always have different hash values when produced, even if they'd be identical otherwise after the addition of padding.

SHA-256 is part of the SHA-2 family of cryptographic hash functions formally specified in:
- [National Institute of Standards and Technology (2015). *FIPS PUB 180-4: Secure Hash Standard*.](https://doi.org/10.6028/NIST.FIPS.180-4)
- [*RFC 6234: US Secure Hash Algorithms (SHA and SHA-based HMAC and HKDF)*. Internet Engineering Task Force.](https://tools.ietf.org/html/rfc6234)

### Byte Handling
To start on this task, we first need to read in the file as a collection of bytes. This is farely easy and self explanatory to implement. Right after that we calculate the original content length in bits. 

In [167]:
# Create a file called file.txt with our three bytes in it for example purposes.
file = 'file.txt'

with open(file, 'w') as f: # For testing purposes, write some text to a file.
    f.write('abc')

# Read in the file
with open(file, 'rb') as f:
    data = f.read()

# Calculate length in bits of the file
length = len(data) * 8

print(f'Length of file in bits: {length}')

Length of file in bits: 24


### Apply Padding
Following the length calculation, we should follow the SHA-256 specification for applying padding. First off, I'll add a single '1' bit as a hexidecimal (0x80). We need to add enough '0' bits so the total length of our file (including both the original length, the padding and 64 bits) is a multiple of 512 bits. 

Let me break this down to make it easier to understand: 
- The initial 1 we use indicates where the original message ends and our padding begins. For this code, we represent it as 0x80 (10000000 in binary ).
- We then add the zero padding, enough to bring the total length up to 448 bits modulosed by 512. What this does is it holds on to exactly 64 bits at the end for the length of the message.
- These 64 bits respresent the length of our original message, which helps us ensure different messages with their own lengths have different hashes. 

### Padding Specifications

For more, the padding scheme described is detailed in:
- [National Institute of Standards and Technology (2015). *FIPS PUB 180-4: Secure Hash Standard*, Section 5.1.1: Padding the Message.](https://doi.org/10.6028/NIST.FIPS.180-4)
- [*Introduction to Modern Cryptography* (3rd ed.), Chapter 5: Message Authentication Codes. CRC Press.](https://doi.org/10.1201/9781351133036)

In [168]:
# Add a '1' bit to the end of the data, in hex this would be 0x80 or 10000000 in python.
padding = bytearray([0x80]) 

We use the following calculation to ensure we leave exactly 64 bits (or 8 bytes) at the end of the file. 

In [169]:
### Example Code here
data = b'abc'  # Example data
padding_needed = 56 - ((len(data) + 1) % 64)
if padding_needed < 0:
    padding_needed += 64

Following this, we add back the original length of the file as a 64 bit big-endian unsigned integer. 

In [170]:
# mock values
padding_zeros = padding_needed // 8 
original_length = len(data) * 8 

# Add original length of file in bits, this must be a 64 big-endian unsigned integer. So as to not overflow.
padding.extend([0] * padding_zeros)
padding.extend(original_length.to_bytes(8, byteorder='big'))

### Final Implementation

In [171]:
### Final Sha256 Method

# Calculate the SHA256 padding of a string from a giving file.
def SHA256_padding(file):
    # Read in the file
    with open(file, 'rb') as f:
        data = f.read()

    # Calculate length in bits of the file, multiply by 8 to get bits as there is 8 bits in a byte.
    original_length = len(data) * 8

    # Add a '1' bit to the end of the data, in hex this would be 0x80 or 10000000 in python.
    padding = bytearray([0x80])

    # Add enough '0' bits so the total length is 448 mod 512 bits.
    # So the original length + 1 + padding) % 64 = 56, we need 56 to be the remainder. 
    padding_zeros = 56 - ((len(data) + 1) % 64)
    # if we reach the 56-bye mark, add another block. So we dynamically add padding needed to the file. 
    if padding_zeros < 0:
        padding_zeros += 64. # This is the number of bytes needed to reach 448 mod 512.

    # Add original length of file in bits, this must be a 64 big-endian unsigned integer. So as to not overflow.
    padding.extend([0] * padding_zeros)
    padding.extend(original_length.to_bytes(8, byteorder='big'))


    # Print this padding in hexidecimal. 
    print("SHA256 padding in hexidecimal format: ")
    for i, byte in enumerate(padding):
        print(f'{byte:02x}', end=' ')
        if (i + 1) % 25 == 0: # Print 25 bytes per line for simplicity and readability.
            print()
        if len(padding) % 25 != 0:
            print() # Print a newline if the padding is not a multiple of 25 bytes.
    
    return padding # Return the padding at the end. 

### Verifying it Works

In [172]:
### Example usage with a test file. 
def test_sha256_padding():
    # Create a test file with "abc"
    with open("test_file.txt", "wb") as f:
        f.write(b"abc")
    
    # Calculate and print the padding
    padding = SHA256_padding("test_file.txt")
    return padding

# Test the function
padding = test_sha256_padding()


SHA256 padding in hexidecimal format: 
80 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 

00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
00 

00 
00 
00 
00 
00 
00 
00 
00 
00 
00 
18 


### SHA256 Uses
SHA256 is used across the industry. In digital signitures, its used for verifying the integrity of messages or files. In Blockchain, many cryptocurrencies use SHA256 for proof-of-work checking. Passwords are commonly stored as hashes. Software's often check the integrity of files through the use of checksums with SHA256.

- [*The First 30 Years of Cryptographic Hash Functions and the NIST SHA-3 Competition*. In Topics in Cryptology – CT-RSA 2010.](https://doi.org/10.1007/978-3-642-11925-5_1)
- [*Password Storage Cheat Sheet*.](https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html)

# Task 4: Prime Numbers

For the unafiliated, Prime numbers are natural numbers (greater than 1) that cannot be formed by multiplying two smaller natural numbers. It must have exactly two factors, 1 and itself, no divisibles. They have a storied history, whether it be Euclid who proved that prime numbers are infinite and created one of the two algorithms I'll use in this task, (The sieve of Eratosthenes) or others like Fermat, Euler and Gauss who developed various theorems about primes in the 17th and 18th centuries that are considered fundamental.

- [Euclid's *Elements*, Book IX, Proposition 20 (circa 300 BCE) provides the first known proof that prime numbers are infinite.](https://mathcs.clarku.edu/~djoyce/elements/bookIX/propIX20.html)
- [*Prime Numbers: A Computational Perspective* (2nd ed.). Springer.](https://doi.org/10.1007/0-387-28979-8)

## Division with square roots
The basic process for this algorithm for checking if a number is prime involves finding out if its divisible by any smaller numbers than it. We do this through using its square roots. This is useful in my opinion, as we only need to check if a number divides into our square root of n. This is because: 
- If its a multiple of n, then either a or b must be included in its square root. 

Let me break this down a little more clearly. When checking if a number n is prime, we only need to check the divisibility up to that number's square root. This is because if n = a x b (and they're both integers above 1) then either a) Both are less than or equal to n's square root. Or b) One is below n's square root and the other is above n's square root.

- [Weisstein, E. W. "Prime Number." From MathWorld--A Wolfram Web Resource.](https://mathworld.wolfram.com/PrimeNumber.html)
- [*An Introduction to the Theory of Numbers* (6th ed.). Oxford University Press.](https://global.oup.com/academic/product/an-introduction-to-the-theory-of-numbers-9780199219865)
- [*Number Theory: Volume I: Tools and Diophantine Equations*. Springer.](https://doi.org/10.1007/978-0-387-49923-9)

With this in mind, we know that if a divisor isn't found up to n's square root. Then no divisor exists to be found and the number can only be prime. Here an example of how we'd use this in our own implementation.

In [173]:
import numpy as np
import matplotlib.pyplot as plt
import time

i = 5 # Start at 5, since we've already checked 2 and 3.
n = 1000000 # Set n to a large number for testing.
while i * i <= n: # Check if i squared is less than or equal to n.
    if n % i == 0 or n % (i + 2) == 0: # Check if n can be divided by i or i + 2, return False if so.
        print("False")
    i += 6 # Increment i by 6, since we're checking numbers of the "6k±1" form.
print("True")

False
False
False
False
True


## Using the 6k±1 form

We than proceed to check each number in sequence, adding it to our list if its a prime and leaving it be if it isn't one. We continue this process until we hopefully reach 100 prime numbers. For this implementation, I check only numbers of the form 6k±1. This is because all primes greater than 3 are of that form. If you wish, they can all be expressed as 6k, 6k+1, 6k+2, etc. Numbers of the form 6k+2 and 6k+4 are divisible by 2, while numbers of the form 6k+3 are divisible by 3. 

This means that our potential primes to consider in this instance are of the form 6k+1 or 6k+5, all others are handled already. Note that 6k+5 is actually 6k-1 for the next value of k. 

What does this all mean? Well it cuts out a lot of the effort for us. It reduces the number of divisions we need by about 2/3, as we don't need to check every single odd number when checking for primes. 

- [*Elementary Theory of Numbers* (2nd ed.). North-Holland.](https://www.elsevier.com/books/elementary-theory-of-numbers/sierpinski/978-0-444-86662-5)
- [*Journal of Functional Programming*, 19(1), 95-106.](https://doi.org/10.1017/S0956796808007004)

In [174]:
# We only need to test numbers of the form "6k±1" since all primes higher than a 3 are of this form. 
# In order to do this, we must check the edge cases of 2 and 3 separately. Once we've checked 2 we know its not even.
n = 57 # Example number to check for primality.
if n <= 1: # Check if n is less than or equal to 1, if so, return False.
    print("False")
if n <= 3: # Check if n is less than or equal to 3, if so, return True.
    print("True")
if n % 2 == 0 or n % 3 == 0: # Check if n is divisible by 2 or 3, if so, return False.
    print("False")

False


### Final Implementation

In [175]:
# Test if a number is prime by checking if it is divisible by any number up to its square root. n is the number to test.
def findPrimesUsingSquareRoots(n):

    # We only need to test numbers of the form "6k±1" since all primes higher than a 3 are of this form. 
    
    # In order to do this, we must check the edge cases of 2 and 3 separately. Once we've checked 2 we know its not even.
    if n <= 1: # Check if n is less than or equal to 1, if so, return False.
        return False
    if n <= 3: # Check if n is less than or equal to 3, if so, return True.
        return True
    if n % 2 == 0 or n % 3 == 0: # Check if n is divisible by 2 or 3, if so, return False.
        return False
    
    # Check all numbers of the form "6k±1" up to the square root of n.
    i = 5 # Start at 5, since we've already checked 2 and 3.
    while i * i <= n: # Check if i squared is less than or equal to n.
        if n % i == 0 or n % (i + 2) == 0: # Check if n can be divided by i or i + 2, return False if so.
            return False
        i += 6 # Increment i by 6, since we're checking numbers of the "6k±1" form.
    return True

Here is a simple function for running that algorithm a number of times equal to the user's desire.

In [176]:
def find_primes(count):
    # Find the first x number of primes based on the count given.
    primes = [] # Create an empty list to store our found primes.
    num = 2 # Start at 2, the first prime number.

    # Loop until we have found the desired number of primes.
    while len(primes) < count:
        if findPrimesUsingSquareRoots(num):
            primes.append(num)
        num += 1

    return primes

In [177]:
primes = find_primes(100)

print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


## Sieve Algorithm
The previous method works overall, but it's a somewhat brute force approach to the issue in my opinion. Instead we could explore using [the Sieve approach](https://www.jstor.org/stable/106053). Instead of testing every individual number as we progress through sequentially, we can eliminate the composite numbers systematically, allowing us to whittle it down to the 100 primes we need. 

The Sieve of Eratosthenes was developed by the Greek mathematician by the same name in the 3rd century BCE, which makes it incredibly old as far as algorithms go. But that doesn't change its effectiveness in my opinion. 

### How it works

Before we go through this step by step. Here is a summary of how this will work: 
1. We start with a list of numbers, we'll choose a minimum and a maximum of 2 and 1000.
2. Starting with the first prime (2), we mark all its multiples as composite.
3. We moved to the next unmarked number (which must be prime) and we mark all of its multiples
4. Repeat this until the square root of the limit is reached. 
5. By the end, all unmarked numbers should be our primes.

- ["Linear prime-number sieves: a family tree." *Science of Computer Programming*, 9(1), 17-35.](https://doi.org/10.1016/0167-6423(87)90026-9)

### Seive Set up
To start, we'll set our limit and the sieve list. This will be a list containing all the true values found up to the limit we set, in this case 1000. Since the first two values can't be primes, we'll set them to false immediately.

In [178]:
limit = 1000 # Set a limit for the sieve.

# Initialize the sieve list.
sieve = [True] * (limit + 1) # Create a list of True values, one for each number up to the limit.
# Set the first two values to False, since they are not prime.
sieve[0] = False 
sieve[1] = False

So at first, we'll assume all numbers are primes. Then as we loop through them all, we'll mark all the multiples of each prime as false or not prime. Any numbers that aren't marked by this are found as primes. This approach is quite a bit more effective and efficient in comparison to our last algorithm. We only need to search up to a certain limit, because larger factors of a prime would have been marked by their smaller counterparts already. Additionally, each number is marked only once for each of its prime numbers, rather than repeatedly parsing over familiar numbers within the previous algorithm.

In [179]:
# Loop through the sieve and set all multiples of each number to False.
for i in range(2, int(np.sqrt(limit)) + 1): # Loop through up to square root of limit.
    if sieve[i]: # If the number is prime.
        for j in range(i * i, limit + 1, i): # Loop through all multiples of the number.
            sieve[j] = False # Set the multiple to False.

After we've found all our primes using this method, we'll collect all the primes together by looping through the sieve and adding them systematically.

In [180]:
# Loop through the sieve and add all prime numbers to the list.
count = 100 # Set the count of primes to find.
primes = [] # Create empty list to store found primes.
for i in range(limit + 1):
    if sieve[i]: # If number is prime.
        primes.append(i) # Add number to list.
        if len(primes) == count: # Once we reach count, break loop.
            break

print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Final Implementation
Here is the final function combining each of these steps for finding primes with the sieve method. As mentioned, we initialise some set up, loop through and systematically remove the multiples of each number, and then collect our primes that remain together and return it. 

In [181]:
def findPrimesUsingSieve(count):
    # Find the first x number of primes based on the count given.
    limit = 1000 # Set a limit for the sieve.

    # Initialize the sieve list.
    sieve = [True] * (limit + 1) # Create a list of True values, one for each number up to the limit.
    # Set the first two values to False, since they are not prime.
    sieve[0] = False 
    sieve[1] = False

    # Loop through the sieve and set all multiples of each number to False.
    for i in range(2, int(np.sqrt(limit)) + 1): # Loop through up to square root of limit.
        if sieve[i]: # If the number is prime.
            for j in range(i * i, limit + 1, i): # Loop through all multiples of the number.
                sieve[j] = False # Set the multiple to False.

    # Loop through the sieve and add all prime numbers to the list.
    primes = [] # Create empty list to store found primes.
    for i in range(limit + 1):
        if sieve[i]: # If number is prime.
            primes.append(i) # Add number to list.
            if len(primes) == count: # Once we reach count, break loop.
                break

    return primes


print(findPrimesUsingSieve(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


In [182]:
# Compare the results to ensure they match
print(f"\nBoth algorithms produce the same results: {find_primes(100) == findPrimesUsingSieve(100)}")


Both algorithms produce the same results: True


## Comparing algorithms
The Sieve algorithm seems far more effective at first glance, particularly in terms of speed. This is generally due to the fact that it eliminates multiple composite numbers with each of its loops, instead of individually. It marks off all the multiples of the known primes, avoiding redundancy. As you can imagine, this is especially effective for larger numbers that possess many factors. Sieve has a handy amorphous nature to it as well, working across the entire range of numbers rather than singularly. If we were to directly compare time complexity between both methods, we'd find the following: 

- Square Root method Time Complexity: O(n√n).
- Sieve method Time Complexity: O(n log log n).

There are some additional trade offs to be considered when comparing these though, lets break it down: 

### Square Root Algorithm Breakdown: 
- Simple to understand and easier to implement.
- Arguably more efficient for scenarios where we're testing individual numbers.
- Uses minimal memory.

However, its less efficient for generating sequences of primes and considerably slower.

### Sieve of Eratosthenes Breakdown: 
- Far more efficient for generating a series primes up to a limit, like this task requires.
- Faster by a considerable margin.

However, it requires more memory (depending on the limit used) and not practical for cases where our range is incredibly large, because of this memory restraint. This means in select cases where resources are limited, the Square Root approach may actually be your better option in my opinion.

- [Knuth, D. E. (1997). *The Art of Computer Programming, Volume 2: Seminumerical Algorithms* (3rd ed.). Addison-Wesley. Section 4.5: Factoring Into Primes.](https://www-cs-faculty.stanford.edu/~knuth/taocp.html)
- ["Trading time for space in prime number sieves." *Lecture Notes in Computer Science*, 1423, 179-195.](https://doi.org/10.1007/BFb0054863)

## Prime Number Uses
Prime numbers are used across the technology industry today, here's a couple *prime* examples: 
- As mentioned previously, hashing makes use of primes in hash table sizing to reduce the chances of collisions.
- In cryptography, RSA and other public-key systems use factoring large numbers since they often prove difficult to do.
- number generation can utilise primes to aid in creating high-quality sequences of random numbers.




- ["A method for obtaining digital signatures and public-key cryptosystems." *Communications of the ACM*, 21(2), 120-126.](https://dl.acm.org/doi/10.1145/359340.359342)
- [*Introduction to Algorithms* (3rd ed.). MIT Press. Section 11.3: Hash Functions.](https://mitpress.mit.edu/books/introduction-algorithms-third-edition)
- [*Handbook of Applied Cryptography*. CRC Press. Chapter 4: Public-Key Parameters.](https://cacr.uwaterloo.ca/hac/)
- ["TestU01: A C library for empirical testing of random number generators." *ACM Transactions on Mathematical Software*, 33(4), Article 22.](https://doi.org/10.1145/1268776.1268777)

# Task 5: Roots

So our aim with this task is to take the first 100 prime numbers (which we've gathered in our previous task) and calculate the first 32 bits of eah of their fractional parts of their square roots. We can do this similarly to how the SHA-256 algorithm generates its [constants](https://www.wiley.com/en-us/Applied+Cryptography%3A+Protocols%2C+Algorithms+and+Source+Code+in+C%2C+20th+Anniversary+Edition-p-9781119096726) (Add link here). 

### Cryptographic Constants
For constant generation, the designers of SHA-256 used the fractional elements of cube roots of the first 64 prime numbers for making their round constants (also known as k values) and the fractional parts of square roots of initial 8 primes for their hash values (also known as H values).
Why do these constants matter in Cryptography and computational theory? Well the unpredictability is key in my opinion, the designers needed to make sure that no patterns or exploits could be found for attackers to make use of. They're also unbiased in nature, so they aren't specially chosen to reveal weaknesses. Lastly is this: they appear random but they're actually deterministic. You can verify where they came from. This is why the fractional parts are viable for our approach here, they provide us with set of numbers that for all intents and purposes look random, but take a closer look and you'll see they're actually derived from classic maths principles.

### Square Roots
First off, lets get the square root of each prime number. To do this in python, you multiply against the power of 0.5 of your prime. As shown below, for example purposes I've used 7 as its a fairly common prime number. Using the exponentiation operator with 0.5 gives us the square root, in this case 2.6457513110645907.

- [*An Introduction to the Theory of Numbers* (6th ed.). Oxford University Press. Section 4.5: Irrational Numbers.](https://global.oup.com/academic/product/an-introduction-to-the-theory-of-numbers-9780199219865)

In [183]:
prime = 7
root = prime ** 0.5 # Square root is written as a power of 0.5 in Python.
print(f'root: {root}')

root: 2.6457513110645907


## Fractional Parts
After this we need to extract the fractional parts of our square root. The fractional parts is (as the name suggests) anything after the decimal point. So for the square root of 7: 2.6457513110645907, it'd be 6457513110645907. To extract the fractional part of our square rooted prime in python, I think my best option is to use math.floor and take that away from our root, that way I only get the fractional part. 

- [*What every computer scientist should know about floating-point arithmetic*. ACM Computing Surveys, 23(1), 5-48. ](https://doi.org/10.1145/103162.103163)

In [184]:
frac = root - math.floor(root) # Subtract the floor of the root from the root to get the fractional part.
print(f'frac: {frac}')

frac: 0.6457513110645907


### 32 Bits Representation
Next up is where we dive into bits territory. In this case, we need to represent our fractional part (such as .6457513110645907) as a 32 bit value (like ). We can do this by multiplying our fractional part from above by 2 to the power of 32. What this does is it effectively shifts the decimal point 32 digits over to the right, giving us our desired 32 bits of a fractional part.

Lastly, I'll use int() just to truncate down the found result to keep it within integer format. 

- [Knuth, D. E. (1997). *The Art of Computer Programming, Volume 2: Seminumerical Algorithms* (3rd ed.). Addison-Wesley. Section 4.2.1: Floating Point Arithmetic.](https://www-cs-faculty.stanford.edu/~knuth/taocp.html)

In [185]:
frac = frac * (2 ** 32) # Multiply the fractional part by 2^32 to get the fractional part in integer form.
print(f'frac: {frac}')

# Convert to integer through truncation so we only have the integer part.
bits = int(frac)
print(f'bits: {bits}')

frac: 2773480762.37154
bits: 2773480762


### Find Square Root - Method
Now that we've accomplished this algorithm step by step. I'll combine this together into a singular method.

In [186]:
def findSquareRoot(prime):
    root = prime ** 0.5 # Square root is written as a power of 0.5 in Python.

    frac = root - math.floor(root) # Subtract the floor of the root from the root to get the fractional part.

    frac = frac * (2 ** 32) # Multiply the fractional part by 2^32 to get the fractional part in integer form.
    
    bits = int(frac) # Convert to integer through truncation so we only have the integer part.
    return bits


### The Final Product
Now I'm going to feed in my primes calculated in the previous task using one of my algorithms into this algorithm we've developed and collect the results.

In [187]:
primes = findPrimesUsingSieve(100) # Find the first 100 primes using the sieve method, alternatively I could use the other method.

# calculate fractional parts of the square roots
frac32Results = []
for prime in primes:
    bits = findSquareRoot(prime)
    frac32Results.append(bits)

# Print results
print("First 32 bits of fractional parts of square roots of first 100 primes:")
for prime, bits in zip(primes, frac32Results):
    print(f"{prime:6} → {bits:032b}")

First 32 bits of fractional parts of square roots of first 100 primes:
     2 → 01101010000010011110011001100111
     3 → 10111011011001111010111010000101
     5 → 00111100011011101111001101110010
     7 → 10100101010011111111010100111010
    11 → 01010001000011100101001001111111
    13 → 10011011000001010110100010001100
    17 → 00011111100000111101100110101011
    19 → 01011011111000001100110100011001
    23 → 11001011101110111001110101011101
    29 → 01100010100110100010100100101010
    31 → 10010001010110010000000101011010
    37 → 00010101001011111110110011011000
    41 → 01100111001100110010011001100111
    43 → 10001110101101000100101010000111
    47 → 11011011000011000010111000001101
    53 → 01000111101101010100100000011101
    59 → 10101110010111111001000101010110
    61 → 11001111011011001000010111010011
    67 → 00101111011100110100011101111101
    71 → 01101101000110000010011011001010
    73 → 10001011010000111101010001010111
    79 → 11100011011000001011010110010110
    8

### Why this Approach?
There's several reasons really. The square root of any prime number is irrational in nature. This means it cannot be expressed as a fraction of integers. What this does is it ensures the bits that result from it have no simple predictable pattern.
Another reason is distribution, the binary digits of these numbers distribute evenly, meaning that roughly each 0 and 1 occur equally. There's just enough variance in their probability to avoid obvious patterns forming. On a simpler side, independence plays a reason to use them. The bits we get from different square roots of prime numbers show no correlation between each other. 

calculating the first 32 bits like we are in this task has several practical uses of benefit I think. We could use them for getting values for generating random numbers or for ID creation, where our IDs wouldn't have any patterns that were obvious, keeping them secure. Complementary to what we explored in task 2, we could make use of this for crafting our own hash functions for cryptography.

Outside of SHA-256, this approach is used in other algorithms for a similar cryptographic task. SHA-1 at one point used specific constants, but later on it moved to the square/cube root method seen here. MD5 uses similar but with sine functions instead. ChaCha uses select initial words from an ASCII string (expand 32-byte k").

- [*Cryptography Engineering: Design Principles and Practical Applications*. Wiley. Chapter 5: Hash Functions.](https://www.wiley.com/en-us/Cryptography+Engineering%3A+Design+Principles+and+Practical+Applications-p-9780470474242)

# Task 6: Proof of Work

This task is an approximation of the "Proof of Work" mechanism. We can see this used in cryptocurrencies and Bitcoin. Its used often to [decide who gets to add the next block to a blockchain](https://bitcoin.org/bitcoin.pdf). It aids in distribution, providing a way to divide and seperate out who gets coins fairly. Its also a security feature, it makes attempts at network attacks expensive to do. There's several things to consider when doing this task that I mentioned in previous ones that make this a difficult process in my opinion. The Avalache Effect plays into this, where its rone to completely different hash output if even the tiniest changes are made to the input. The Even Distributions play into this too, where we have roughly 50% zeros and 50% ones. Lastly is that this is kind of one-way function. Calculating the hash from the input data isn't terribly difficult, but its far more difficult to find the input when you only have the hash.

### Download English words
I used the file of english words used in the previous module for this task. While it may not be the largest catalogue of words, it proved sufficient for the task. Additionally, other files I found online possessed numerous unprovable words, so I felt best to use this one instead. 

In [188]:
import hashlib
import requests
import os
import time
from collections import defaultdict

def getEnglishDict():
    englishDict = "words.txt" # Path to the word list file.


    # Download the word list if it doesn't exist
    if not os.path.exists(englishDict):
        print("Downloading English Dict...")
        url = "https://github.com/ShanewalshGit/EmergingTech/blob/main/tasks/words.txt?raw=true"
        response = requests.get(url)
        with open(englishDict, "w") as f:
            f.write(response.text)
        print("Download complete.")
    
    # Read the word list
    with open(englishDict, "r") as f:
        words = [line.strip() for line in f if line.strip()]
    
    return words
    
# Only prints some of the words to avoid cluttering the output.
print("First 5 words in the dictionary:")
for word in getEnglishDict()[:5]:
    print(word)

print("Last 5 words in the dictionary:")
for word in getEnglishDict()[-5:]:
    print(word)


First 5 words in the dictionary:
AARHUS
AARON
ABABA
ABACK
ABAFT
Last 5 words in the dictionary:
ZOROASTER
ZOROASTRIAN
ZULU
ZULUS
ZURICH


### Count Leading Zeros in a Hash
So to do this we need to look at the binary version of each of our hashes, going from left to right. As we do this, we count up all zeros until we inevitably hit a 1. There's a little more work prior to this however, namely calculating the hash of a word and converting that to binary. Its vital as well that we don't have '0b' there as a prefix, so we take that out else our counting method will break at 'b'.

- [*Cryptographic Hash-Function Basics: Definitions, Implications, and Separations for Preimage Resistance, Second-Preimage Resistance, and Collision Resistance*. In Fast Software Encryption (pp. 371-388). Springer.](https://doi.org/10.1007/978-3-540-25937-4_24)

### Hash Calculation
Here's how we could handle the collection of the hash digest for a word. Later on, I'll mix this together with the greater code as the first step. I'll let the method handle one word individually for this process, and loop the method at the end instead of handling all the words in the one method call.

In [189]:
# calculate the hash (SHa256)
word = "hello"
hash = hashlib.sha256(word.encode())
hash = hash.hexdigest()

### Binary Conversion
Here's how I could convert the hex gained from our hash into a sequence of binaries. As seen below, we remove the '0b' prefix from it.

In [190]:
# Convert hex to string of binaries (we don't want '0b' on there)
binary = bin(int(hash, 16))[2:].zfill(256)

### Count Leading Zeros
Counting the number of leading zeros present is simple enough for us. For each bit in the binary string we gained previously, we check if the bit is a zero. If so, increase our count or stop the count otherwise.

In [191]:
# Count up leading zeros
leadingZeros = 0
for bit in binary: 
    if bit == '0':
        leadingZeros += 1 # add up 1 if bit found to be a 0
    else:
        print(f"Leading zeros: {leadingZeros}")

Leading zeros: 2
Leading zeros: 3
Leading zeros: 3
Leading zeros: 5
Leading zeros: 5
Leading zeros: 5
Leading zeros: 5
Leading zeros: 7
Leading zeros: 9
Leading zeros: 11
Leading zeros: 11
Leading zeros: 12
Leading zeros: 12
Leading zeros: 13
Leading zeros: 13
Leading zeros: 13
Leading zeros: 14
Leading zeros: 16
Leading zeros: 17
Leading zeros: 17
Leading zeros: 17
Leading zeros: 17
Leading zeros: 17
Leading zeros: 17
Leading zeros: 18
Leading zeros: 18
Leading zeros: 22
Leading zeros: 23
Leading zeros: 26
Leading zeros: 26
Leading zeros: 30
Leading zeros: 30
Leading zeros: 30
Leading zeros: 33
Leading zeros: 35
Leading zeros: 35
Leading zeros: 36
Leading zeros: 36
Leading zeros: 36
Leading zeros: 37
Leading zeros: 42
Leading zeros: 42
Leading zeros: 42
Leading zeros: 43
Leading zeros: 43
Leading zeros: 45
Leading zeros: 46
Leading zeros: 47
Leading zeros: 48
Leading zeros: 48
Leading zeros: 51
Leading zeros: 52
Leading zeros: 52
Leading zeros: 53
Leading zeros: 53
Leading zeros: 53
L

### Counting Zeros Execution

In [192]:
# Calculate SHA256 hash of word and count leading zeros amount.
def countLeadingZeros(word):

    # calculate the hash (SHa256)
    hash = hashlib.sha256(word.encode())
    hash = hash.hexdigest()

    # Convert hex to string of binaries (we don't want '0b' on there)
    binary = bin(int(hash, 16))[2:].zfill(256)

    # Count up leading zeros
    leadingZeros = 0
    for bit in binary: 
        if bit == '0':
            leadingZeros += 1 # add up 1 if bit found to be a 0
        else:
            break

    return word, leadingZeros, hash

### Calculate Word With Most Leading Zeros
Here I'll develop a function for finding the collection of words with the most leading zeros (using our function we just wrote) and extract the top ones for showcasing.

In [193]:
# Find the words that have the most leading zeros in their SHA256 hash
def findWordsWithLeadingZeros(wordsList, resultsNum=10, maxWords=1000000):
    
    print(f"Searching through {maxWords} words for the top {resultsNum} with the most leading zeros in their SHA256 hash...")

    # Keep track of words
    result = defaultdict(list)
    maxZeros = 0 # Keep track of max zeros


    # Search words (limited to maxWords for efficiency)
    for i, word in enumerate(wordsList[:maxWords]): # Enumerate through list of words.
        word, zeros, hash = countLeadingZeros(word) # Count leading zeros for each word.
        
        # Keep track of max zeros and add to results
        if zeros >= maxZeros:
            max_zeros = zeros
            result[zeros].append((word, zeros, hash)) # Add word to results.

    # Get top 10 results
    topResults = []
    for zeros in sorted(result.keys(), reverse=True):
        topResults.extend(result[zeros])
        if len(topResults) >= resultsNum:
            break

    return topResults[:resultsNum]


### Execution
Now to execute our functionality that returns the top 10 words with the most leading zeros in their hash. We find that "applicant" has the most at 16 zeros with a marginal lead over "anachronistically" at 15. Then we have "neck" at 14 and "dryly" at 13 and so on. Our results continue from there down to "poisons" at 12 zeros. This gives us an interesting insights I feel. The words with the most leading zeros are quite marginal in there difference, at most there is a 4 zero difference between the highest and the 10th highest.

In [194]:
words = getEnglishDict() # Get the English dictionary words
print(f"Loaded {len(words)} English words") # Print number of words loaded.

# Find top words with most leading zeros in their SHA256 hash
top_words = findWordsWithLeadingZeros(words)

print("\nTop words with most leading zero bits in SHA256 hash:")
print("=" * 70) # For formatting purposes.
print(f"{'Word':<20} {'Zero Bits':<10} {'SHA256 Hash'}") # Print a header.
print("-" * 70) # For formatting purposes.

for word, zeros, hash in top_words: # Print the top words with leading zeros.
    print(f"{word:<20} {zeros:<10} {hash[:16]}...") # Print the word, zeros, and first 16 characters of the hash.



Loaded 45402 English words
Searching through 1000000 words for the top 10 with the most leading zeros in their SHA256 hash...

Top words with most leading zero bits in SHA256 hash:
Word                 Zero Bits  SHA256 Hash
----------------------------------------------------------------------
APPLICANT            16         0000ca01adc973c2...
ANACHRONISTICALLY    15         000116c1be4890e0...
NECK                 14         00020b4f21dc1d0b...
DRYLY                13         0006dca1a7f786ba...
METAMORPHOSIS        13         0005fa7f4fd53079...
PLATOON              13         0004124cd544662f...
SURGERY              13         00042315571e0259...
BERESFORD            12         000e6a1da3ecda87...
INTERFACED           12         000850ba098df06a...
POISONS              12         000d45e96eb28889...


### Proof word is in a Dictionary
For the sake of referencing and sanctity, here are sources for each of the top 10 words found. I've included proof of word existence for each of the top 10 using both merriam-webster and Oxford dictionary links for each of them. 

In [195]:
def showProof(word):

    print("\nDictionary proof:")
    print(f"Local: ")
    print(f"The word '{word}' is included in the words.txt dictionary.")

    # Online site alternatives for greater proof if my txt ins't enough.
    print(f"Online Proof: ")
    print(f"- Merriam-Webster: https://www.merriam-webster.com/dictionary/{word.lower()}")
    print(f"- Oxford: https://www.oxfordlearnersdictionaries.com/definition/english/{word.lower()}")

# Verify the top word(s) are in a dictionary
for word, zeros, hash in top_words:
    showProof(word)


Dictionary proof:
Local: 
The word 'APPLICANT' is included in the words.txt dictionary.
Online Proof: 
- Merriam-Webster: https://www.merriam-webster.com/dictionary/applicant
- Oxford: https://www.oxfordlearnersdictionaries.com/definition/english/applicant

Dictionary proof:
Local: 
The word 'ANACHRONISTICALLY' is included in the words.txt dictionary.
Online Proof: 
- Merriam-Webster: https://www.merriam-webster.com/dictionary/anachronistically
- Oxford: https://www.oxfordlearnersdictionaries.com/definition/english/anachronistically

Dictionary proof:
Local: 
The word 'NECK' is included in the words.txt dictionary.
Online Proof: 
- Merriam-Webster: https://www.merriam-webster.com/dictionary/neck
- Oxford: https://www.oxfordlearnersdictionaries.com/definition/english/neck

Dictionary proof:
Local: 
The word 'DRYLY' is included in the words.txt dictionary.
Online Proof: 
- Merriam-Webster: https://www.merriam-webster.com/dictionary/dryly
- Oxford: https://www.oxfordlearnersdictionaries.

## Proof of Work Uses
Other than cryptocurrencies: proof of work is used in Anti-spam functionality in email systems. This is where it requires email senders to solve small proof of work challenges that help [prevent mass spam](https://doi.org/10.1016/j.jisa.2020.102716), this akin to the classic little site minigames you do to prove you're not a robot. Proof of work is also used for [DDoS protection](https://www.cloudflare.com/learning/ddos/what-is-a-ddos-attack/) to filter out automated attacks from various services. APIs make use of proof of work all the time for rate limiting the allocation of resources.

# Task 7: Turing Machines

For this task, I'll design a Turing Machine that adds 1 to a binary sequence.

## Computation with Turing
The concept of the Turing machine originates from the 1936 paper ["On Computable Numbers, with an Application to the Entscheidungsproblem."](https://doi.org/10.1112/plms/s2-42.1.230) This relates back to David Hilbert who posed the question: Is there an algorithm that can determine whether a given maths statement is provable? This was Turings answer to that. The machine he created was simple at first glance, surprisingly so in my opinion. But it was more than powerful enough to do what it set out to. It could represent any processes for algorithms it needed to. It predated the invention of physical elctronic computers as we know them by years. He coined the term "algorithm" for the actual first time in this work. In his aim to answer Hilbert's question, he proved that many problems were fundametally unsolvable by any possible algorithm.  


For the purposes of this task, our turing machine will use the following elements: 
- A tape, usually divided into cells, each one contains a symbol from a set collection
- A "head" that reads, writes or updates the symbols in cells
- A register that stores current state from a set of available states
- A transition system that decides what to do next based on current state and symbol

When it comes to the states handling, we'll need to initialise our initial state that the computational process begins in, the acceptance state that tells us process has been succesful and a rejection state that tell us its failed or halted the computational process.

So for this approach, we need to to add 1 to the binary sequence. We start at the left-most symbol and should slowly move to the right side (the least significant bit) or in other words, the right-most symbol.
This is then proceeded by a different change based on the current bit's state: 
- If the bit is 0, change it to 1
- if the bit is 1, change it to 0 and carry the 1 to the left

In [196]:
# R: Move right towards end
# A: Add or carry state for incrementing
# L: Move left towards start
# H: Halt when complete, stop incrementing

Q = {'R', 'A', 'L', 'H'} # Our set of states

# Tape alphabet, the symbols we can read/write
T = {'0', '1', '_'} # _ represents blank space in the tape.

b = '_' # Initialize blank space

# Input alphabet, the symbols we can read/write that aren't blank
A = {'0', '1'}

# Set initial state
s = 'R'

# Set final state
F = {'H'}



So for this, I'll set up a function that defines the rules for this turing machine, so as we can accurately simulate it in Python. This is typically called a transition or increment table. 

In [197]:
def TuringIncrementTable():
    return {
        # State R, move right
        ('R', '0'): ('R', '0', 'R'),  # If 0, keep moving right
        ('R', '1'): ('R', '1', 'R'),  # If 1, keep moving right
        ('R', '_'): ('A', '_', 'L'),  # If end, move left to Least significant bit (rightmost digit) and switch to Add state
        
        # State A, add or carry
        ('A', '0'): ('L', '1', 'L'),  # Change 0 to 1, no carry needed, move left
        ('A', '1'): ('A', '0', 'L'),  # Change 1 to 0, continue carry, move left
        ('A', '_'): ('L', '1', 'L'),  # If we carried past digit fathest to left, add a 1
        
        # State L, move left
        ('L', '0'): ('L', '0', 'L'),  # If 0, keep moving left
        ('L', '1'): ('L', '1', 'L'),  # If 1, keep moving left
        ('L', '_'): ('H', '_', 'R'),  # Reached beginning, halt machine
    }

With thse rules in mind, lets develop a function that will simulate a turing machine. This will add 1 to a binary number as prescribed. It'll have numerous arguments. Namely, the transition table itself that defines our rules, an initial binary input passed in as a string and "verbose" bool for stating whether we should or shouldn't print each step. The last argument is more for debugging and displaying the process than it is for use in the actual technical implementation.

In [198]:
def simulateTuringMachine(transitionTable, initialBinary, verbose=True): 
    if verbose: 
        print(f"Initial binary: {initialBinary}")

    # Handle empty input
    if initialBinary == '':
        initialBinary = '_' # If empty, set to blank space.

    # Initialise tape and its position
    tape = list(initialBinary)
    position = 0 # Start to 0, the left most position of tape.

    # Set Initial state
    state = 'R' # Start in R state, the 'move right' state.

    # Count operations performed
    operations = 0

    if verbose:
        print(f"Starting configuration: {state}{''.join(tape)}") # Print starting configuration of tape.

    # Loop until we reach halt state
    while state != 'H':
        # Ensure position is valid (within bounds of tape)
        if position < 0: # If we go past left end of tape, add a blank space to left.
            tape.insert(0, '_')
            position = 0
        elif position >= len(tape): # If we go past right end of tape, add a blank space to right.
            tape.append('_')

        # Get current symbol under tape head
        symbol = tape[position]

        # Get transition for current state and symbol, according to transition table.
        if (state, symbol) in transitionTable:
            nextState, nextSymbol, direction = transitionTable[(state, symbol)]

            # Update tape and state according to transition
            tape[position] = nextSymbol
            state = nextState

            # Move tape head in specified direction
            if direction == 'L':
                position -= 1
            else: # Move right
                position += 1

            operations += 1 # Increment operations count

            if verbose: 
                # Print current configuration - state shown at current position
                config = ''.join(tape[:position]) + state + ''.join(tape[position:]) # Combine tape into string with state in the middle.
                print(f"Step {operations}: {config}") 

            else:
                # No transition found, halt machine
                if verbose:
                    print(f"No transition defined for state={state}, symbol={symbol}")
                break
        
        # Clean up any leading/trailing underscores or in our case blanks
        while tape and tape[0] == '_':
            tape.pop(0) # Remove leading blanks

        result = ''.join(tape)

        if verbose:
            print(f"Final result after adding 1: {result}")
            print(f"Total operations: {operations}")
        
    return result


So first the machine starts in state R. Here it will start moving entirely to the right in an effort to find the least significant bit (which is the rightmost digit). This is a simple enough, it just moves right continuously until it hits a blank symbol.

Now we move into state A, this is where it starts adding or carrying bits. Once its at the end, it moves left to the least significant bit, if its a 0, it changes it to a 1. It then proceeds to move into state L, denoting that it is done with addition. If it sees a 1 however, it'll change it to a 0 and continue on leftwards as it carries the 1 with it. Note as well that if it ever needs to carry farther than the leftmost digit it'll add a new 1 at that position.

Once done with addition, since its in state L, it'll move left towards the beginning. Eventually it finishes in state H (Halt) once it reaches there, it will denote this by a blank symbol to its left.

For testing purposes, here is a set of test cases to run it through, including the initial example one mentioned. As seen when ran, the turing machine succesfully traverses the binary string per the transition table and reaches the expected outcome. 

In [206]:
# Test with the example: 100111 -> 101000
def testTuringIncrement():
    # set the transition table
    transitionTable = TuringIncrementTable()
    
    # Test cases - the example from task plus a couple of edge cases.
    testCases = [
        "100111",  # 100111 + 1 = 101000
        "0",       # 0 + 1 = 1
        "1",       # 1 + 1 = 10
        "11",      # 3 + 1 = 4 (11 + 1 = 100)
        "1111",    # 15 + 1 = 16 (1111 + 1 = 10000)
    ]
    
    for test in testCases:
        print("TEST CASE START")
        print(f"Testing binary increment on: {test}")
        result = simulateTuringMachine(transitionTable, test, verbose=True)
        print(f"INPUT: {test}, RESULT: {result}")
        
    return "All tests completed"

# Run the tests
testTuringIncrement()

TEST CASE START
Testing binary increment on: 100111
Initial binary: 100111
Starting configuration: R100111
Step 1: 1R00111
Final result after adding 1: 100111
Total operations: 1
Step 2: 10R0111
Final result after adding 1: 100111
Total operations: 2
Step 3: 100R111
Final result after adding 1: 100111
Total operations: 3
Step 4: 1001R11
Final result after adding 1: 100111
Total operations: 4
Step 5: 10011R1
Final result after adding 1: 100111
Total operations: 5
Step 6: 100111R
Final result after adding 1: 100111
Total operations: 6
Step 7: 10011A1_
Final result after adding 1: 100111_
Total operations: 7
Step 8: 1001A10_
Final result after adding 1: 100110_
Total operations: 8
Step 9: 100A100_
Final result after adding 1: 100100_
Total operations: 9
Step 10: 10A0000_
Final result after adding 1: 100000_
Total operations: 10
Step 11: 1L01000_
Final result after adding 1: 101000_
Total operations: 11
Step 12: L101000_
Final result after adding 1: 101000_
Total operations: 12
Step 13: 10

'All tests completed'

## Turing Machine Limitations
There are some [limitations to what the Turing Machine](https://doi.org/10.1112/plms/s2-43.6.544) can do, in which Alan Turing himself proved. These include: 
- Halting problem, figuring out will any possible program eventually stop.
- Entscheidungsproblem problem, figuring if a mathematical statement is provable.
- Rice's Theorem, any non-trivial property about the behaviour of programs.

## Turing Machine Uses
The principles of a Turing Machine are very clearly foundational to much of computational work today. The principles it brought are all over, such as: 
- [Compiler designs](https://www.pearson.com/us/higher-education/program/Aho-Compilers-Principles-Techniques-and-Tools-2nd-Edition/PGM167067.html) were influenced by state machines for analysis of lexicons. 
- Regular Expressions are essentially [limited versions of Turing Machines](https://doi.org/10.1145/363347.363387).
- Algorithm Analysis: Turing Machines are fundamental for understand computational complexity (explored more in next task), algorithms as a whole and their origins.




# Task 8: Computational Complexity

## Computational Complexity Theory
This is a branch of Computer Science many would consider foundational. It studies resources like time and space required to solve different computational problems. It emerged in the 60s and 70s with the work of researchers like Richard Karp, Michael Rabin and Stephen Cook who at the time, established several structured frameworks that allowed for analysing algorithms and their efficiency. The field of Computational Complexity aims to tackle questions like "How efficiently can we solve this problem?" or "What limitations does this algorithm have?" - [*The complexity of theorem-proving procedures*. In Proceedings of the Third Annual ACM Symposium on Theory of Computing](https://doi.org/10.1145/800157.805047)

For a lot of this, we use what known as Asymptotic notation, this is where Big O notation comes from, along with [Omega and Theta](https://doi.org/10.1145/1008328.1008329). As an example of how these systems are used, Bubble Sort (which we're about to implement) typically has a time complexity of O(n²) at the average to worst cases, but in the best cases scenario it'll manage O(n), but this is only when the array is already sorted. For space complexity, it has O(1) because it simply sorts in-place. - [*Computers and Intractability: A Guide to the Theory of NP-Completeness*](http://www.csc.kth.se/~viggo/problemlist/compendium.html)

## What is Bubble Sort?

Bubble sort is a well known algorithm across programming. For the unfamilliar, it works by stepping through a list of numbers or units repeatedly and comparing their adjacent elements, we then swap them if they're order is incorrect. Its name derives from the fact that elements slowly "bubble" to the top of the list as the iterations sort it. Bubble sort isn't always the most efficient choice for larger datasets, but I find its easily understandable and effective to implement. Its memory overhead is minimal enough and great for learning to understand sorting algorithms. - [*The Art of Computer Programming, Volume 3: Sorting and Searching*](https://www-cs-faculty.stanford.edu/~knuth/taocp.html)

### Array Traversal
The first step to accomplishing a general implementation of bubble sort is the traversal process. We should always start at the very beginning of our array. We compare its one adjacent element (to the right). If our initial is greater than the second for example, they are swapped. 

### Comparisons or "Bubbling" Process

After that is the "bubbling" step so to speak. This is a continuation of before. So we progress each time through each pair of adjacent elements until we reach the end. Each pair swapping if necessary as they go, After one entire pass through, our largest element of the list should now be at the top.. or else something has gone horribly wrong. 

In [200]:
# Snippit code of bubbling up process
def bubbleUp(arr):
    arrLength = len(arr)
    
    print(f"Initial array: {arr}")
    
    # Single pass demonstration
    for j in range(arrLength-1):
        # Compare adjacent elements
        if arr[j] > arr[j+1]:
            # Swap if they are in the wrong order
            print(f"  Swap {arr[j]} and {arr[j+1]}")
            arr[j], arr[j+1] = arr[j+1], arr[j]
            print(f"  Array after swap: {arr}")
        else:
            print(f"  No swap needed for {arr[j]} and {arr[j+1]}")
    
    print(f"After one pass: {arr}")
    print(f"Notice how the largest element ({max(arr)}) has bubbled up to the end!")
    
    return arr

# Example usage
sample_array = [5, 1, 4, 2, 3]
bubbleUp(sample_array)

Initial array: [5, 1, 4, 2, 3]
  Swap 5 and 1
  Array after swap: [1, 5, 4, 2, 3]
  Swap 5 and 4
  Array after swap: [1, 4, 5, 2, 3]
  Swap 5 and 2
  Array after swap: [1, 4, 2, 5, 3]
  Swap 5 and 3
  Array after swap: [1, 4, 2, 3, 5]
After one pass: [1, 4, 2, 3, 5]
Notice how the largest element (5) has bubbled up to the end!


[1, 4, 2, 3, 5]

In [201]:
# Snippit code of bubbling down process
def bubbleDown(arr):
    arrLength = len(arr)
    
    print(f"Initial array: {arr}")
    
    # Single pass demonstration (starting from the end)
    for j in range(arrLength-1, 0, -1):
        # Compare adjacent elements
        if arr[j] < arr[j-1]:
            # Swap if they are in the wrong order
            print(f"  Swap {arr[j]} and {arr[j-1]}")
            arr[j], arr[j-1] = arr[j-1], arr[j]
            print(f"  Array after swap: {arr}")
        else:
            print(f"  No swap needed for {arr[j]} and {arr[j-1]}")
    
    print(f"After one pass: {arr}")
    print(f"Notice how the smallest element ({min(arr)}) has bubbled down to the beginning!")
    
    return arr

# Example usage
sample_array = [5, 3, 4, 2, 1]
bubbleDown(sample_array)

Initial array: [5, 3, 4, 2, 1]
  Swap 1 and 2
  Array after swap: [5, 3, 4, 1, 2]
  Swap 1 and 4
  Array after swap: [5, 3, 1, 4, 2]
  Swap 1 and 3
  Array after swap: [5, 1, 3, 4, 2]
  Swap 1 and 5
  Array after swap: [1, 5, 3, 4, 2]
After one pass: [1, 5, 3, 4, 2]
Notice how the smallest element (1) has bubbled down to the beginning!


[1, 5, 3, 4, 2]

### Subsequent Passes
After this things are pretty self explanatory, the algorithm repeats the previous process for the remaining elements until there's no longer any incorrect positioning. This of course, ignores the elements sorted to the end in previous passes.

In [202]:
# Snippit code of subsequent passes
def visualizePasses(arr):
    arrLength = len(arr)
    
    print(f"Initial array: {arr}")
    
    # Traverse through all array elements
    for i in range(arrLength):
        # Track if any swaps were made in this pass
        swapped = False
        
        print(f"\nPass {i+1}:")
        print(f"  Starting: {arr}")
        
        # Last i elements are already in place
        for j in range(0, arrLength-i-1):
            # Compare adjacent elements
            if arr[j] > arr[j+1]:
                # Swap if they are in the wrong order
                print(f"  Swap {arr[j]} and {arr[j+1]}")
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True
        
        print(f"  After pass {i+1}: {arr}")
        
        # If no swapping occurred in this pass, array is sorted
        if not swapped:
            print(f"  No swaps in this pass - array is sorted!")
            break
    
    return arr

# Example usage
sample_array = [5, 1, 4, 2, 8]
visualizePasses(sample_array)

Initial array: [5, 1, 4, 2, 8]

Pass 1:
  Starting: [5, 1, 4, 2, 8]
  Swap 5 and 1
  Swap 5 and 4
  Swap 5 and 2
  After pass 1: [1, 4, 2, 5, 8]

Pass 2:
  Starting: [1, 4, 2, 5, 8]
  Swap 4 and 2
  After pass 2: [1, 2, 4, 5, 8]

Pass 3:
  Starting: [1, 2, 4, 5, 8]
  After pass 3: [1, 2, 4, 5, 8]
  No swaps in this pass - array is sorted!


[1, 2, 4, 5, 8]

### Optimization
A minor addition I tried here was using a swapped boolean to give this algorithm the ability to stop early if nothing was swapped in an entire pass through the array. This implies its already sorted, so we can abort the process early. 

### Final Product

In [203]:
def bubbleSort(arr): # Takes in a list of numbers to sort.
    arrLength = len(arr) # Get the length of array.

    # Print the initial array for debugging purposes.
    print(f"Initial array: {arr}, length: {arrLength} \n")

    # Traverse through the array, for each pass.
    # The outer loop represents the number of passes through the array, while the inner loop represents the comparisons made in each pass.
    # The outer loop runs for the length of the array, while the inner loop runs for the length of the array minus the current pass number.
    for i in range(arrLength):
        swapped = False # Set swapped to False at start of each pass.
        for j in range(0, arrLength - i - 1):
            if arr[j] > arr[j + 1]:
                # Swap if element found is greater than next element
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        # Print array state after a pass, for debugging purposes.
        print(f"Pass {i+1}: {arr}")
        
        # If no swapping occurred in pass, array is sorted
        if not swapped:
            break
            
    return arr

# Initial test of our bubble sort function
arr = [64, 34, 25, 12, 22, 11, 90]
print("Unsorted array:", arr)
sorted_arr = bubbleSort(arr)
print("\nSorted array:", sorted_arr)



Unsorted array: [64, 34, 25, 12, 22, 11, 90]
Initial array: [64, 34, 25, 12, 22, 11, 90], length: 7 

Pass 1: [34, 25, 12, 22, 11, 64, 90]
Pass 2: [25, 12, 22, 11, 34, 64, 90]
Pass 3: [12, 22, 11, 25, 34, 64, 90]
Pass 4: [12, 11, 22, 25, 34, 64, 90]
Pass 5: [11, 12, 22, 25, 34, 64, 90]
Pass 6: [11, 12, 22, 25, 34, 64, 90]

Sorted array: [11, 12, 22, 25, 34, 64, 90]


### Various examples
Here are some general examples of the bubble sort in operation. Including already sorted arrays and reversed ones being passed into the method. 

In [204]:
# Test with some examples
test_array1 = [64, 34, 25, 12, 22, 11, 90]
print("Sorting test_array1:")
bubbleSort(test_array1)

print("\nSorting test_array2:")
test_array2 = [5, 1, 4, 2, 8]
bubbleSort(test_array2)

# Test with an already sorted array
print("\nSorting already sorted array:")
test_array3 = [1, 2, 3, 4, 5]
bubbleSort(test_array3)

# Test with a reverse sorted array
print("\nSorting reverse sorted array:")
test_array4 = [5, 4, 3, 2, 1]
bubbleSort(test_array4)

Sorting test_array1:
Initial array: [64, 34, 25, 12, 22, 11, 90], length: 7 

Pass 1: [34, 25, 12, 22, 11, 64, 90]
Pass 2: [25, 12, 22, 11, 34, 64, 90]
Pass 3: [12, 22, 11, 25, 34, 64, 90]
Pass 4: [12, 11, 22, 25, 34, 64, 90]
Pass 5: [11, 12, 22, 25, 34, 64, 90]
Pass 6: [11, 12, 22, 25, 34, 64, 90]

Sorting test_array2:
Initial array: [5, 1, 4, 2, 8], length: 5 

Pass 1: [1, 4, 2, 5, 8]
Pass 2: [1, 2, 4, 5, 8]
Pass 3: [1, 2, 4, 5, 8]

Sorting already sorted array:
Initial array: [1, 2, 3, 4, 5], length: 5 

Pass 1: [1, 2, 3, 4, 5]

Sorting reverse sorted array:
Initial array: [5, 4, 3, 2, 1], length: 5 

Pass 1: [4, 3, 2, 1, 5]
Pass 2: [3, 2, 1, 4, 5]
Pass 3: [2, 1, 3, 4, 5]
Pass 4: [1, 2, 3, 4, 5]
Pass 5: [1, 2, 3, 4, 5]


[1, 2, 3, 4, 5]

## Time Complexity of Bubble Sort
As mentioned previously, Bubble Sort on average has a Time Complexity of O(n²), if we compare this against other sorting algorithms... its not ideal. Merge Sort for comparison is O(n log n), Quick Sort and selection sort are both O(n²) aswell at their worst, but they're not as stable as Bubble Sort. Our work here does highlight something pretty important which is worst and average cases vs best cases. While theoretically, our performance could be O(n) with Bubble Sort at its best, this is rarely the case and in actual practical scenarios, we cannot rely on it being the case. Using different examples above shows us how it behaves in different instances or degrees of sorted. The examples take a variety of passes to sort, varying quite a bit based on their input. - [*Introduction to the Design and Analysis of Algorithms* (3rd ed.). Pearson. Chapter 3: Brute Force and Exhaustive Search.](https://www.pearson.com/us/higher-education/program/Levitin-Introduction-to-the-Design-and-Analysis-of-Algorithms-3rd-Edition/PGM229455.html)

## Some Background
Bubble sort is one of the classic older sorting algorithms that is still used heavily to this day. It is however, considered quite inefficient for larger datasets as discussed above. Why do we still use it? Well its thought to be easy to implement and understand, making it very useful for teaching purposes. The code needed is minimal enough, conceptually its very easy to wrap your head around and it uses little memory to operate. Its also very stable, preserving the relative order of equal elements.

- [*History of sorting algorithms*](https://doi.org/10.1145/800025.1198367)
- [*Sorting networks and their applications*](https://doi.org/10.1145/1468075.1468121)

## Bubble Sort Uses
Even though it isn't neccessarally the most efficient sorting algorithm, it still finds uses across the industry today. - [*Data Structures, Algorithms, and Applications in Java*](https://www.cise.ufl.edu/~sahni/dsaaj/)
- Smaller datasets make great use of it. When n is on the smaller side, it can be far more competitive of an option. 
- Its very efficient for data that is almost sorted, its best case scenario of O(n) makes it efficient for this.
- As discussed it has great use in educational settings for teaching about sorting algorithms and computational complexity. I for one can back up that it has had an impact researching it on my understanding of Time Complexity as a concept. 
- Its easy to implement, which for many cases can be of benefit. In terms of space complexity, since its an in-place algorithm, it can be easier for lots of hardware to implement.
- When memory and resources are limited on a platform, bubble sort can be far more valuable with O(1) space complexity.